In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import  DecisionTreeClassifier,export_graphviz
from xgboost import plot_importance
from matplotlib import pyplot as plt
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

### 查看模型参数

In [121]:
clf.fit(X,labels)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0.5, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.3, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=1, max_depth=9,
              max_leaves=None, min_child_weight=0.5, missing=nan,
              monotone_constraints=None, n_estimators=190, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

## 内部测试后，用ts1数据实际预测

In [122]:
X1 = ts1.drop(['pax_name','emd_lable2'],axis=1).values
X1 = mm.fit_transform(X1)
X1

array([[0.3097454 , 0.13346457, 0.47956467, ..., 0.        , 0.        ,
        1.        ],
       [0.34012033, 0.1003937 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.55910806, 0.18858268, 0.74143282, ..., 0.        , 0.        ,
        0.43284145],
       ...,
       [0.37384004, 0.12755906, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.3837384 , 0.1515748 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.20874945, 0.20984252, 0.        , ..., 0.        , 0.        ,
        0.        ]])

## 进行预测

In [123]:
prediction2=clf.predict(X1)
prediction2

array([0, 0, 0, ..., 1, 0, 0])

In [124]:
prediction3=clf.predict_proba(X1)
prediction3

array([[9.9497724e-01, 5.0227679e-03],
       [5.2945900e-01, 4.7054097e-01],
       [9.9926364e-01, 7.3638186e-04],
       ...,
       [4.0718484e-01, 5.9281516e-01],
       [7.8884149e-01, 2.1115853e-01],
       [5.2945900e-01, 4.7054097e-01]], dtype=float32)

In [125]:
prediction_=np.array(prediction3[:,1])

# 将预测值和测试数据索引构造表，输出csv上传，查看成绩

In [126]:
t = pd.DataFrame({'pax_name': ts1.pax_name, 'emd_lable2': prediction2,'probability':prediction_})
t

,pax_name,emd_lable2,probability
0,21f0b1c838160ac26cb2c57660bc3fd5,0,0.005023
1,21e0621a85f6db6139ff7cf2d53b4e5d,0,0.470541
2,197b215b23a93b19f391c422eb27f310,0,0.000736
3,92b435f20c6ce2fd5acef7b6ff49b6d0,1,0.604254
4,a38ac8bfe0feb14c0469e2917bc74a05,1,0.592815
...,...,...,...
6766,d6364954adcfbd58d8beebd686a9fbdc,0,0.470541
6767,150b405be4282907eb7e87f992856ea1,0,0.470541
6768,33b02fadd0d05938b8cc3dfbc735fd10,1,0.592815
6769,29cb3dea4ecce778096484ff2e097f5b,0,0.211159


In [127]:
pclass = pd.crosstab(t.emd_lable2,t.emd_lable2,margins=True)
pclass

emd_lable2,0,1,All
emd_lable2,,,
0,4926,0,4926
1,0,1845,1845
All,4926,1845,6771


In [136]:
t = t[t['emd_lable2'].isin([1])]
t = t.sort_values(by='probability',ascending=False)
t1 = t.head(500)
t2 = t.head(6771)
t1
t2

,pax_name,emd_lable2,probability
6113,279f3b7a538187e2971d8c877abf7ca6,1,0.953778
6244,307d1dd1c1ae4be1603ef669a7b094b4,1,0.882011
6428,eaa1bb76299b2848f16de272dfd9fcc0,1,0.791577
4220,9193384ef490f9a93db516f631f365c0,1,0.730130
627,e7489c6a1c3273141d9640aa7e7c8647,1,0.730130
...,...,...,...
2549,80d717f3a26db16fcb013e121c908fce,1,0.592815
2611,b6024ed359b042f8e70a6087316757ab,1,0.592815
2545,25b17daffbd115b6d838c8c36e0f1ce4,1,0.592815
2607,4f665a9d9a40fa962b0edf9ada047874,1,0.592815


# 导出表格

In [138]:
t1.to_csv('D:/专业实习/Xg_class-master/data/result.csv',index=False)
t2.to_csv('D:/专业实习/Xg_class-master/data/all.csv',index=False)